In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
import hashlib
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [5]:
%%time
#train_identity
train_identity = dd.read_csv('../input/train_identity.csv')
train_identity = train_identity.compute()
train_identity = train_identity.reset_index(drop=True)
print("train_identity finished")
#test_identity
test_identity = dd.read_csv('../input/test_identity.csv')
test_identity = test_identity.compute()
test_identity = test_identity.reset_index(drop=True)
print("test_identity finished")
#train_transaction
train_transaction = dd.read_csv('../input/train_transaction.csv')
train_transaction = train_transaction.compute()
train_transaction = train_transaction.reset_index(drop=True)
print("train_transaction finished")
#test_transaction
test_transaction = dd.read_csv('../input/test_transaction.csv')
test_transaction = test_transaction.compute()
test_transaction = test_transaction.reset_index(drop=True)
print("test_transaction finished")
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(sample_submission.shape)

train_identity finished
test_identity finished
train_transaction finished
test_transaction finished
(506691, 2)
CPU times: user 1min 9s, sys: 11.2 s, total: 1min 20s
Wall time: 27.7 s


In [6]:
print(train_identity.shape,test_identity.shape,train_transaction.shape,test_transaction.shape)

(144233, 41) (141907, 41) (590540, 394) (506691, 393)


In [7]:
%%time
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

CPU times: user 7.85 s, sys: 7.03 s, total: 14.9 s
Wall time: 14.8 s


In [8]:
train.head()

TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5   card6  addr1  addr2  dist1  dist2  \
0    NaN  150.0    discover  142.0  credit  315.0   87.0   19.0    NaN   
1  404.0  150.0  mastercard  102.0  credit  325.0   87.0    NaN    NaN   
2  490.0  150.0        visa  166.0   debit  330.0   87.0  287.0    NaN   
3  567.0  150.0  mastercard  117.0   debit  476.0   87.0    NaN    NaN   
4  514.0  150.0  mastercard  102.0  credit  420.0   87.0    NaN    NaN   

  P_emaildomain R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  \
0           NaN           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
1     gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
2   outlook.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
3     yahoo.com           NaN  2.0  5.0  0.0  0.0  0.0  4.0  0.0  0.0  1.0   
4     gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0   

   C10  C11  C12   C13  C14     D1     D2    D3    D4   D5  D6  D7  D8  D9  \
0  0.0  2.0  0.0   1.0  1.0   14.0    NaN  13.0   NaN  NaN NaN NaN NaN NaN   
1  0.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN   0.0  NaN NaN NaN NaN NaN   
2  0.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN   0.0  NaN NaN NaN NaN NaN   
3  0.0  1.0  0.0  25.0  1.0  112.0  112.0   0.0  94.0  0.0 NaN NaN NaN NaN   
4  1.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN   NaN  NaN NaN NaN NaN NaN   

    D10    D11  D12  D13  D14    D15   M1   M2   M3   M4   M5   M6   M7   M8  \
0  13.0   13.0  NaN  NaN  NaN    0.0    T    T    T   M2    F    T  NaN  NaN   
1   0.0    NaN  NaN  NaN  NaN    0.0  NaN  NaN  NaN   M0    T    T  NaN  NaN   
2   0.0  315.0  NaN  NaN  NaN  315.0    T    T    T   M0    F    F    F    F   
3  84.0    NaN  NaN  NaN  NaN  111.0  NaN  NaN  NaN   M0    T    F  NaN  NaN   
4   NaN    NaN  NaN  NaN  NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

    M9   V1   V2   V3   V4   V5   V6   V7   V8   V9  V10  V11  V12  V13  V14  \
0  NaN  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  1.0   
2    F  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0   
3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V15  V16  V17  V18  V19  V20  V21  V22  V23  V24  V25  V26  V27  V28  V29  \
0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V30  V31  V32  V33  V34  V35  V36  V37  V38  V39  V40  V41  V42  V43  V44  \
0  0.0  0.0  0.0  0.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0   
2  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0   
3  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V45  V46  V47  V48  V49  V50  V51  V52  V53  V54  V55  V56  V57  V58  V59  \
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0  1.0  0.0  0.0  0.0   
1  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  

In [9]:
used_columns = ['card1','card2','card3','card5','addr1','addr2']

In [10]:
train[used_columns].isnull().sum()

card1        0
card2     8933
card3     1565
card5     4259
addr1    65706
addr2    65706
dtype: int64

In [11]:
for i in tqdm(used_columns):
    train[i].fillna(0,inplace=True)
    test[i].fillna(0,inplace=True)    

100%|██████████| 6/6 [00:00<00:00, 342.13it/s]


In [12]:
used_columns = ['card4','card6']

In [13]:
for i in tqdm(used_columns):
    train[i].fillna('nan',inplace=True)
    test[i].fillna('nan',inplace=True)    

100%|██████████| 2/2 [00:00<00:00, 26.03it/s]


In [14]:
def card_info_hash(x):
    s = (str(int(x['card1']))+
         str(int(x['card2']))+
         str(int(x['card3']))+
         str(x['card4'])+
         str(int(x['card5']))+
         str(x['card6']))
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h

In [15]:
def device_hash(x):
    s =  str(x['id_30'])+str(x['id_31'])+str(x['id_32'])+str(x['id_33'])+str( x['DeviceType'])+ str(x['DeviceInfo'])
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h

In [16]:
train['card_hash'] = train.apply(lambda x: card_info_hash(x), axis=1   )
train['device_hash'] = train.apply(lambda x: device_hash(x), axis=1   )

In [17]:
test['card_hash'] = test.apply(lambda x: card_info_hash(x), axis=1   )
test['device_hash'] = test.apply(lambda x: device_hash(x), axis=1   )

In [18]:
print(len(set(train['card_hash'])),len(set(train['device_hash'])))

14893 9815


In [19]:
train_1 = train[train['isFraud'] ==1]
train_1 = train_1[['TransactionID','isFraud','card_hash','device_hash']].reset_index(drop=True)
print(train_1.shape)

(20663, 4)


In [20]:
train_1.head()

TransactionID  isFraud        card_hash      device_hash
0        2987203        1  53be1a94a5a5750  c7f2e00e03ae096
1        2987240        1  832c88c03f01511  7e44a5ac4b72dc7
2        2987243        1  832c88c03f01511  7e44a5ac4b72dc7
3        2987245        1  832c88c03f01511  7e44a5ac4b72dc7
4        2987288        1  12e54d765cf4f88  60b8c05b45ee5d6

In [21]:
temp = train_1.groupby(['card_hash','device_hash'])['isFraud'].count().reset_index(drop=False)
print(temp.shape)

(6111, 3)


In [22]:
temp.head()

card_hash      device_hash  isFraud
0  0019f3763312db0  0cc3f1aaedf763c        1
1  0019f3763312db0  1fdfefa8326f295        1
2  0019f3763312db0  3e7811d2d010233        1
3  0019f3763312db0  714756cd8079a81        1
4  0019f3763312db0  ca952d77cd443d4        2

In [23]:
temp[temp['isFraud'] > 20].shape

(143, 3)

In [24]:
temp = temp[temp['isFraud'] > 20].reset_index(drop=True)

In [25]:
temp['merged_hash'] = temp['card_hash'] + '_' + temp['device_hash']

In [26]:
temp.head()

card_hash      device_hash  isFraud                      merged_hash
0  0089e971022bd9f  0124ee4825bf29f       37  0089e971022bd9f_0124ee4825bf29f
1  0089e971022bd9f  61c20f9f84db8b0       28  0089e971022bd9f_61c20f9f84db8b0
2  0127340322320a7  c7f2e00e03ae096       30  0127340322320a7_c7f2e00e03ae096
3  02089e4201c478a  c7f2e00e03ae096      250  02089e4201c478a_c7f2e00e03ae096
4  023f5d33b724cb0  c7f2e00e03ae096      141  023f5d33b724cb0_c7f2e00e03ae096

In [27]:
print(len(set(temp['merged_hash'])))

143


In [28]:
temp = list(temp['merged_hash'])

In [29]:
test['merged_hash'] = test['card_hash'] + '_' + test['device_hash']

In [30]:
train['merged_hash'] = train['card_hash'] + '_' + train['device_hash']

In [31]:
test['merged_hash_in'] = test['merged_hash'].progress_apply(lambda x: 1 if x in temp else 0)

100%|██████████| 506691/506691 [00:01<00:00, 360404.28it/s]


In [32]:
train['merged_hash_in'] = train['merged_hash'].progress_apply(lambda x: 1 if x in temp else 0)

100%|██████████| 590540/590540 [00:01<00:00, 367015.44it/s]


In [33]:
print(train['merged_hash_in'].sum(),test['merged_hash_in'].sum())

187902 147734


In [34]:
train_less= train[train['merged_hash_in'] ==1]

In [35]:
train_less['merged_hash'].value_counts()

e7cb958bc9fcdbe_c7f2e00e03ae096    13258
02089e4201c478a_c7f2e00e03ae096     9240
d2c3243d549807e_c7f2e00e03ae096     8771
49cb22f326df6fe_c7f2e00e03ae096     6707
9ee8e589bec241e_c7f2e00e03ae096     6552
3dcac3668b3812d_c7f2e00e03ae096     6112
9a461887c830051_c7f2e00e03ae096     6047
6f473e021469c90_c7f2e00e03ae096     5487
d2ca229767cb03d_c7f2e00e03ae096     4733
7f85e7defc9a181_c7f2e00e03ae096     3897
815ea97bc6e30f8_c7f2e00e03ae096     3521
ed704c9a30d5185_c7f2e00e03ae096     3518
22de74414367853_c7f2e00e03ae096     3493
0610e14ca7f6c50_c7f2e00e03ae096     3480
41e70506bdbc666_c7f2e00e03ae096     3416
e5218c4a6ba6b0d_c7f2e00e03ae096     3330
0cac5612d2f3305_c7f2e00e03ae096     3271
7c2005cac0dd50f_c7f2e00e03ae096     3231
2911521a2650174_c7f2e00e03ae096     2951
b2e5dc2deea675f_c7f2e00e03ae096     2864
13fe9a2373b8fc2_c7f2e00e03ae096     2860
697e04bfea34575_c7f2e00e03ae096     2701
7f985ed6bafd564_c7f2e00e03ae096     2598
354027dc5af1138_c7f2e00e03ae096     2572
1a0ca926cc19011_

In [36]:
temp2 = pd.concat([train_less.groupby('merged_hash')['isFraud'].mean().sort_values(ascending=False),train_less['merged_hash'].value_counts()],axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [37]:
print(temp2.shape)

(143, 2)


In [38]:
temp2.head()

isFraud  merged_hash
0089e971022bd9f_0124ee4825bf29f  0.093199          397
0089e971022bd9f_61c20f9f84db8b0  0.136585          205
0127340322320a7_c7f2e00e03ae096  0.037453          801
02089e4201c478a_c7f2e00e03ae096  0.027056         9240
023f5d33b724cb0_c7f2e00e03ae096  0.077473         1820

In [39]:
temp2.columns = ['mean_','count_']

In [40]:
temp2 = temp2.sort_values(by = 'mean_',ascending=False)

In [41]:
temp2

mean_  count_
f8360028f0c401d_8c77e8689f54c7e  1.000000      57
f8360028f0c401d_4682f7ab67f956b  1.000000     112
620227200c64fb9_abda5e39116a69d  1.000000      21
a2bcb561fce2b69_57839ffc2020545  1.000000      21
7086d86140f07a4_c7f2e00e03ae096  1.000000      46
42c398e0ff9f923_6b24aeb6313e820  1.000000      24
751777fefa9891d_669f9256b04fb02  1.000000      41
b00a5853e414876_0f8eda5b168c6aa  1.000000      38
72b54c868f8ba62_7980bccc521e983  0.925926      27
4a2342b84d32b77_03181861db4399b  0.780000      50
433a14108fca3fe_c7f2e00e03ae096  0.766667      30
f8360028f0c401d_0124ee4825bf29f  0.709677      31
543212993b885f8_c7f2e00e03ae096  0.694444      36
bbdccfb754b3c0c_c7f2e00e03ae096  0.619048      42
a7ce5ed4c1e6fbb_c7f2e00e03ae096  0.617021      47
9acf0177b0a1d05_c7f2e00e03ae096  0.611111      36
93b299ca0114561_c7f2e00e03ae096  0.604651      43
72b54c868f8ba62_5cab701228a8111  0.586207      87
72b54c868f8ba62_ca5ab7e2b4dc566  0.575758      66
029663ace7c77d6_c7f2e00e03ae096  0.572917      96
13daf93a5aed502_c7f2e00e03ae096  0.543478      46
33dda198f64f21e_c7f2e00e03ae096  0.526882      93
0a43d9e1f08ad3d_c7f2e00e03ae096  0.487500     160
2736189784e2158_c7f2e00e03ae096  0.443182      88
bdea661517f4d0f_c7f2e00e03ae096  0.410811     185
42c398e0ff9f923_3a963895bcfc399  0.387097      62
1a6e0499a586df3_0124ee4825bf29f  0.340206      97
ba335500048094d_5cab701228a8111  0.310811      74
42c398e0ff9f923_03181861db4399b  0.305882      85
046089c4e6ece42_c7f2e00e03ae096  0.300000      90
5692f3e9a31226d_c7f2e00e03ae096  0.279762     168
a2bcb561fce2b69_0124ee4825bf29f  0.274725      91
196fc3b72f0cf45_ca5ab7e2b4dc566  0.266667     105
42c398e0ff9f923_ca5ab7e2b4dc566  0.262295     122
d1996568725cb0f_c7f2e00e03ae096  0.256173     324
ddcb679b0621c40_c7f2e00e03ae096  0.245763     118
ba335500048094d_0124ee4825bf29f  0.221239     113
d0795d17afcb500_c7f2e00e03ae096  0.210227     176
a7014db66fda379_5cab701228a8111  0.198758     161
31e6fd11d440865_0124ee4825bf29f  0.198276     116
196fc3b72f0cf45_0124ee4825bf29f  0.197970     197
a2206c490b8e901_0124ee4825bf29f  0.196850     127
af0f97a38822afa_c7f2e00e03ae096  0.194118     170
4d19476e097ed2a_c7f2e00e03ae096  0.191176     136
71d64b83af4c1aa_5cab701228a8111  0.190476     147
42c398e0ff9f923_5cab701228a8111  0.190244     205
8dc1f0b5812f72b_c7f2e00e03ae096  0.183267     251
4d0cc0aae61ed96_61c20f9f84db8b0  0.172619     168
c0e206aaead2389_c7f2e00e03ae096  0.169065     278
72d283ea4844aff_c7f2e00e03ae096  0.163551     214
42eafa9d861e10f_c7f2e00e03ae096  0.142857     224
42c398e0ff9f923_050fe59058079be  0.141935     155
21746359e5cf055_c7f2e00e03ae096  0.137996     529
0089e971022bd9f_61c20f9f84db8b0  0.136585     205
4d0cc0aae61ed96_0124ee4825bf29f  0.136364     374
4d0cc0aae61ed96_c7f2e00e03ae096  0.122642     318
37993b1e348b1d6_c7f2e00e03ae096  0.120513     780
3226102ae35575d_c7f2e00e03ae096  0.118644     177
9a0e5a6e6dd9a21_c7f2e00e03ae096  0.116883     231
53b9627f0a38e6f_c7f2e00e03ae096  0.108491     212
f2c3c2de908a477_5cab701228a8111  0.106996     243
0089e971022bd9f_0124ee4825bf29f  0.093199     397
a06e6a294218ee9_c7f2e00e03ae096  0.089202     426
196fc3b72f0cf45_c7f2e00e03ae096  0.088028     284
c4587c70055f3cd_c7f2e00e03ae096  0.087291     779
a6d98df513ff711_c7f2e00e03ae096  0.083117     385
bf9464b6a19cb29_c7f2e00e03ae096  0.082621     351
76b000af07b813b_c7f2e00e03ae096  0.080235     511
023f5d33b724cb0_c7f2e00e03ae096  0.077473    1820
751777fefa9891d_22c78ef1347397e  0.076087     460
265722c13b5ec07_c7f2e00e03ae096  0.068282     454
eab7804a303ce6f_c7f2e00e03ae096  0.067831     973
e5218c4a6ba6b0d_c7f2e00e03ae096  0.062462    3330
fa490dde711c809_c7f2e00e03ae096  0.062016     516
751777fefa9891d_5cab701228a8111  0.060561     677
72494f2bed0140f_c7f2e00e03ae096  0.053812    1338
db18c28e00d487d_c7f2e00e03ae096  0.052917     737
f8360028f0c401d_c7f2e00e03ae096  0.051471     408
996132db54dcf02_c7f2e00e03ae096  0.051225     449
9c33db33cc88d7f_c7f2e00e03ae096  0.0

In [42]:
absolutely_zero = list(temp2.tail(10).index)
absolutely_one = list(temp2.head(25).index)

In [43]:
print(absolutely_zero)

['0cac5612d2f3305_c7f2e00e03ae096', 'd2ca229767cb03d_c7f2e00e03ae096', '6f473e021469c90_c7f2e00e03ae096', 'ed704c9a30d5185_c7f2e00e03ae096', '815ea97bc6e30f8_c7f2e00e03ae096', '13fe9a2373b8fc2_c7f2e00e03ae096', '41e70506bdbc666_c7f2e00e03ae096', '9a461887c830051_c7f2e00e03ae096', '7f985ed6bafd564_c7f2e00e03ae096', 'd2c3243d549807e_c7f2e00e03ae096']


In [44]:
print(absolutely_one)

['f8360028f0c401d_8c77e8689f54c7e', 'f8360028f0c401d_4682f7ab67f956b', '620227200c64fb9_abda5e39116a69d', 'a2bcb561fce2b69_57839ffc2020545', '7086d86140f07a4_c7f2e00e03ae096', '42c398e0ff9f923_6b24aeb6313e820', '751777fefa9891d_669f9256b04fb02', 'b00a5853e414876_0f8eda5b168c6aa', '72b54c868f8ba62_7980bccc521e983', '4a2342b84d32b77_03181861db4399b', '433a14108fca3fe_c7f2e00e03ae096', 'f8360028f0c401d_0124ee4825bf29f', '543212993b885f8_c7f2e00e03ae096', 'bbdccfb754b3c0c_c7f2e00e03ae096', 'a7ce5ed4c1e6fbb_c7f2e00e03ae096', '9acf0177b0a1d05_c7f2e00e03ae096', '93b299ca0114561_c7f2e00e03ae096', '72b54c868f8ba62_5cab701228a8111', '72b54c868f8ba62_ca5ab7e2b4dc566', '029663ace7c77d6_c7f2e00e03ae096', '13daf93a5aed502_c7f2e00e03ae096', '33dda198f64f21e_c7f2e00e03ae096', '0a43d9e1f08ad3d_c7f2e00e03ae096', '2736189784e2158_c7f2e00e03ae096', 'bdea661517f4d0f_c7f2e00e03ae096']


In [45]:
train[train['merged_hash'].isin(absolutely_zero)].shape

(44222, 438)

In [46]:
test[test['merged_hash'].isin(absolutely_zero)].shape

(36010, 437)

In [47]:
train[train['merged_hash'].isin(absolutely_one)].shape

(1523, 438)

In [48]:
test[test['merged_hash'].isin(absolutely_one)].shape

(371, 437)

In [52]:
test[test['merged_hash'].isin(absolutely_zero)].shape

(36010, 437)

In [50]:
preds = pd.read_csv('../predictions/pred30.csv')

In [51]:
preds.head()

TransactionID   isFraud
0        3663549  0.000537
1        3663550  0.001017
2        3663551  0.001345
3        3663552  0.002789
4        3663553  0.000842

In [55]:
test_0 = test[test['merged_hash'].isin(absolutely_zero)]
test_0 = list(test_0['TransactionID'])
test_1 = test[test['merged_hash'].isin(absolutely_one)]
test_1 = list(test_1['TransactionID'])

In [56]:
print(len(test_0),len(test_1))

36010 371


In [57]:
def random_func(data):
    a = data['TransactionID']
    b = data['isFraud']
    if a in test_0:
        return 0.0
    elif a in test_1:
        return 1.0
    else:
        return b

In [58]:
preds['isFraud'].min(),preds['isFraud'].max()

(1.6561621805169916e-06, 0.9985324561724112)

In [59]:
preds['isFraud2'] = preds.progress_apply(random_func,axis=1)

100%|██████████| 506691/506691 [55:49<00:00, 151.26it/s]


In [63]:
preds.head()

TransactionID   isFraud  isFraud2
0        3663549  0.000537  0.000537
1        3663550  0.001017  0.001017
2        3663551  0.001345  0.001345
3        3663552  0.002789  0.002789
4        3663553  0.000842  0.000842

In [64]:
del preds['isFraud']
gc.collect()
preds.columns = ['TransactionID','isFraud']

In [68]:
preds.shape

(506691, 2)

In [69]:
sample_submission.shape == preds.shape

True

In [70]:
sample_submission.head()

TransactionID  isFraud
0        3663549      0.5
1        3663550      0.5
2        3663551      0.5
3        3663552      0.5
4        3663553      0.5

In [72]:
sample_submission.dtypes

TransactionID      int64
isFraud          float64
dtype: object

In [71]:
preds.head()

TransactionID   isFraud
0        3663549  0.000537
1        3663550  0.001017
2        3663551  0.001345
3        3663552  0.002789
4        3663553  0.000842

In [73]:
preds.dtypes

TransactionID      int64
isFraud          float64
dtype: object

In [74]:
preds.to_csv('../predictions/pred31.csv',index=False)

In [75]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred31.csv -m "pred31"

100%|███████████████████████████████████████| 13.6M/13.6M [00:33<00:00, 429kB/s]
Successfully submitted to IEEE-CIS Fraud Detection